In [3]:
# Dependencies and Setup
import pandas as pd
import json
import requests
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3
from sqlite3 import Error

In [20]:
# (2016-2017 Season) 
# October 25, 2016 - April 12, 2017
# (2017-2018 Season)
# October 17, 2017 - April 11, 2018
# (2018-2019 Season)
# October 16, 2018 - April 10, 2019

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2016/schedule.json")
schedule_response_2016=schedule_response_obj.json()

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2017/schedule.json")
schedule_response_2017=schedule_response_obj.json()

schedule_response_obj=requests.get("http://data.nba.net/prod/v1/2018/schedule.json")
schedule_response_2018=schedule_response_obj.json()

In [21]:
# Get elements
NBA_Schedule_2016=[]
for item in schedule_response_2016["league"]["standard"]:
    item_dict={}
    item_dict["gameId"] = item["gameId"]
    item_dict["startDateEastern"] = item["startDateEastern"]
    item_dict["hTeam"] = item["hTeam"]["teamId"]
    item_dict["vTeam"] = item["vTeam"]["teamId"]

    # Append List
    NBA_Schedule_2016.append({"gameid":item_dict["gameId"],
                         "gamedate":item_dict["startDateEastern"],
                         "hteam":item_dict["hTeam"],
                         "vteam":item_dict["vTeam"]})

In [22]:
# Get elements
NBA_Schedule_2017=[]
for item in schedule_response_2017["league"]["standard"]:
    item_dict={}
    item_dict["gameId"] = item["gameId"]
    item_dict["startDateEastern"] = item["startDateEastern"]
    item_dict["hTeam"] = item["hTeam"]["teamId"]
    item_dict["vTeam"] = item["vTeam"]["teamId"]

    # Append List
    NBA_Schedule_2017.append({"gameid":item_dict["gameId"],
                         "gamedate":item_dict["startDateEastern"],
                         "hteam":item_dict["hTeam"],
                         "vteam":item_dict["vTeam"]})

In [23]:
# Get elements
NBA_Schedule_2018=[]
for item in schedule_response_2018["league"]["standard"]:
    item_dict={}
    item_dict["gameId"] = item["gameId"]
    item_dict["startDateEastern"] = item["startDateEastern"]
    item_dict["hTeam"] = item["hTeam"]["teamId"]
    item_dict["vTeam"] = item["vTeam"]["teamId"]

    # Append List
    NBA_Schedule_2018.append({"gameid":item_dict["gameId"],
                         "gamedate":item_dict["startDateEastern"],
                         "hteam":item_dict["hTeam"],
                         "vteam":item_dict["vTeam"]})

In [24]:
NBA_Schedule_DF_initial_2016=pd.DataFrame(NBA_Schedule_2016)
NBA_Schedule_DF_initial_2016["gameid"]=NBA_Schedule_DF_initial_2016["gameid"].astype(int)
NBA_Schedule_DF_initial_2016["hteam"]=NBA_Schedule_DF_initial_2016["hteam"].astype(int)
NBA_Schedule_DF_initial_2016["vteam"]=NBA_Schedule_DF_initial_2016["vteam"].astype(int)

NBA_Schedule_DF_initial_2016["gamedate"]=pd.to_datetime(NBA_Schedule_DF_initial_2016['gamedate'])

NBA_Schedule_DF_initial_2016.head()

,gamedate,gameid,hteam,vteam
0,2016-10-01,11600001,1610612761,1610612744
1,2016-10-01,11600002,1610612740,1610612742
2,2016-10-02,11600003,1610612745,12329
3,2016-10-03,11600010,12315,1610612760
4,2016-10-03,11600005,1610612741,1610612749


In [25]:
NBA_Schedule_DF_initial_2017=pd.DataFrame(NBA_Schedule_2017)
NBA_Schedule_DF_initial_2017["gameid"]=NBA_Schedule_DF_initial_2017["gameid"].astype(int)
NBA_Schedule_DF_initial_2017["hteam"]=NBA_Schedule_DF_initial_2017["hteam"].astype(int)
NBA_Schedule_DF_initial_2017["vteam"]=NBA_Schedule_DF_initial_2017["vteam"].astype(int)

NBA_Schedule_DF_initial_2017["gamedate"]=pd.to_datetime(NBA_Schedule_DF_initial_2017['gamedate'])

NBA_Schedule_DF_initial_2017.head()

,gamedate,gameid,hteam,vteam
0,2017-09-30,11700001,1610612744,1610612743
1,2017-09-30,11700002,1610612747,1610612750
2,2017-10-01,11700004,1610612748,1610612737
3,2017-10-01,11700003,1610612761,1610612746
4,2017-10-02,11700005,1610612764,15018


In [26]:
NBA_Schedule_DF_initial_2018=pd.DataFrame(NBA_Schedule_2018)
NBA_Schedule_DF_initial_2018["gameid"]=NBA_Schedule_DF_initial_2018["gameid"].astype(int)
NBA_Schedule_DF_initial_2018["hteam"]=NBA_Schedule_DF_initial_2018["hteam"].astype(int)
NBA_Schedule_DF_initial_2018["vteam"]=NBA_Schedule_DF_initial_2018["vteam"].astype(int)

NBA_Schedule_DF_initial_2018["gamedate"]=pd.to_datetime(NBA_Schedule_DF_initial_2018['gamedate'])

NBA_Schedule_DF_initial_2018.head()

,gamedate,gameid,hteam,vteam
0,2018-09-28,11800001,1610612755,15016
1,2018-09-28,11800002,1610612766,1610612738
2,2018-09-29,11800003,1610612761,1610612757
3,2018-09-29,11800004,1610612742,15021
4,2018-09-29,11800005,1610612744,1610612750


In [55]:
#Load home team_data - 2016
NBA_Schedule_DF_initial_2016_1 = NBA_Schedule_DF_initial_2016[(NBA_Schedule_DF_initial_2016['gamedate'] >= '2016-10-25') & (NBA_Schedule_DF_initial_2016['gamedate'] <= '2017-04-12')]
nba_cols = ["gamedate", "gameid", "hteam"]
NBA_Schedule_DF_initial_2016_1a= NBA_Schedule_DF_initial_2016_1[nba_cols].copy()
NBA_Schedule_DF_initial_2016_1b = NBA_Schedule_DF_initial_2016_1a.rename(columns={"hteam": "teamid"})
NBA_Schedule_DF_initial_2016_1b['home_ind'] = 1

#Load home team_data - 2017
NBA_Schedule_DF_initial_2017_1 = NBA_Schedule_DF_initial_2017[(NBA_Schedule_DF_initial_2017['gamedate'] >= '2017-10-17') & (NBA_Schedule_DF_initial_2017['gamedate'] <= '2018-04-11')]
nba_cols = ["gamedate", "gameid", "hteam"]
NBA_Schedule_DF_initial_2017_1a= NBA_Schedule_DF_initial_2017_1[nba_cols].copy()
NBA_Schedule_DF_initial_2017_1b = NBA_Schedule_DF_initial_2017_1a.rename(columns={"hteam": "teamid"})
NBA_Schedule_DF_initial_2017_1b['home_ind'] = 1

#Load home team_data - 2018
NBA_Schedule_DF_initial_2018_1 = NBA_Schedule_DF_initial_2018[(NBA_Schedule_DF_initial_2018['gamedate'] >= '2018-10-16') & (NBA_Schedule_DF_initial_2018['gamedate'] <= '2019-04-10')]
nba_cols = ["gamedate", "gameid", "hteam"]
NBA_Schedule_DF_initial_2018_1a= NBA_Schedule_DF_initial_2018_1[nba_cols].copy()
NBA_Schedule_DF_initial_2018_1b = NBA_Schedule_DF_initial_2018_1a.rename(columns={"hteam": "teamid"})
NBA_Schedule_DF_initial_2018_1b['home_ind'] = 1

In [56]:
#Load visitor team_data - 2016
NBA_Schedule_DF_initial_2016_2 = NBA_Schedule_DF_initial_2016[(NBA_Schedule_DF_initial_2016['gamedate'] >= '2016-10-25') & (NBA_Schedule_DF_initial_2016['gamedate'] <= '2017-04-12')]
nba_cols = ["gamedate", "gameid", "vteam"]
NBA_Schedule_DF_initial_2016_2a = NBA_Schedule_DF_initial_2016_2[nba_cols].copy()
NBA_Schedule_DF_initial_2016_2b = NBA_Schedule_DF_initial_2016_2a.rename(columns={"vteam": "teamid"})
NBA_Schedule_DF_initial_2016_2b['home_ind'] = 0

#Load visitor team_data - 2017
NBA_Schedule_DF_initial_2017_2 = NBA_Schedule_DF_initial_2017[(NBA_Schedule_DF_initial_2017['gamedate'] >= '2017-10-17') & (NBA_Schedule_DF_initial_2017['gamedate'] <= '2018-04-11')]
nba_cols = ["gamedate", "gameid", "vteam"]
NBA_Schedule_DF_initial_2017_2a = NBA_Schedule_DF_initial_2017_2[nba_cols].copy()
NBA_Schedule_DF_initial_2017_2b = NBA_Schedule_DF_initial_2017_2a.rename(columns={"vteam": "teamid"})
NBA_Schedule_DF_initial_2017_2b['home_ind'] = 0

#Load visitor team_data - 2018
NBA_Schedule_DF_initial_2018_2 = NBA_Schedule_DF_initial_2018[(NBA_Schedule_DF_initial_2018['gamedate'] >= '2018-10-16') & (NBA_Schedule_DF_initial_2018['gamedate'] <= '2019-04-10')]
nba_cols = ["gamedate", "gameid", "vteam"]
NBA_Schedule_DF_initial_2018_2a = NBA_Schedule_DF_initial_2018_2[nba_cols].copy()
NBA_Schedule_DF_initial_2018_2b = NBA_Schedule_DF_initial_2018_2a.rename(columns={"vteam": "teamid"})
NBA_Schedule_DF_initial_2018_2b['home_ind'] = 0

In [57]:
NBA_Schedule_DF = pd.concat([NBA_Schedule_DF_initial_2016_1b,NBA_Schedule_DF_initial_2016_2b, \
                             NBA_Schedule_DF_initial_2017_1b,NBA_Schedule_DF_initial_2017_2b, \
                             NBA_Schedule_DF_initial_2018_1b,NBA_Schedule_DF_initial_2018_2b])

In [61]:
NBA_Schedule_DF.head()

,gamedate,teamid,home_ind
gameid,,,
21600001,2016-10-25,1610612739,1
21600002,2016-10-25,1610612757,1
21600003,2016-10-25,1610612744,1
21600004,2016-10-26,1610612753,1
21600005,2016-10-26,1610612754,1


In [71]:
engine = create_engine("sqlite:///db/NBA_Data.sqlite")
conn = engine.connect()

engine.table_names()

['All_Teams', 'Team_Locations', 'Team_Schedule', 'ranked_2016_2017']

In [70]:
# Set index
#NBA_Schedule_DF.set_index("gameid", inplace=True)

# load final df into sqlite database
NBA_Schedule_DF.to_sql('Team_Schedule2', conn, if_exists='replace', index=True)

In [66]:
pd.read_sql('create table Team_Schedule3 as \
            select a.*, b.teamFullName, b.teamURLcode, TRIM(b.city) as city_name, b.conferenceName, b.divisionName \
            from Team_Schedule2 a \
            join NBA_Teams b on a.teamid=b.teamid', conn)

OperationalError: (sqlite3.OperationalError) no such table: NBA_Teams [SQL: 'create table Team_Schedule_Final as             select a.*, b.teamFullName, b.teamURLcode, TRIM(b.city) as city_name, b.conferenceName, b.divisionName             from Team_Schedule a             join NBA_Teams b on a.teamid=b.teamid'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [67]:
pd.read_sql('select * from Team_Schedule_Final', conn)

OperationalError: (sqlite3.OperationalError) no such table: Team_Schedule_Final [SQL: 'select * from Team_Schedule_Final'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [37]:
pd.read_sql('create table Team_Schedule_Final as select teamid, gamedate, gameid, home_ind, \
            case when home_ind=1 then teamFullName else null end as teamFullName, \
            case when home_ind=1 then teamURLcode else null end as teamURLcode, \
            case when home_ind=1 then city_name else null end as city_name, \
            case when home_ind=1 then conferenceName else null end as conferenceName, \
            case when home_ind=1 then divisionName else null end as divisionName \
            from Team_Schedule', conn)

ResourceClosedError: This result object does not return rows. It has been closed automatically.